In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path
from sklearn.metrics import classification_report
from keras.applications.resnet import ResNet50
# import matplotlib.pyplot as plt
# import seaborn as sns
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
image_dir = Path('data/images')

In [3]:
image_paths = list(image_dir.glob(r'**/*.jpg'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], image_paths))

image_paths_df = pd.Series(image_paths, name='imagepath').astype(str)
labels = pd.Series(labels, name='label')

df = pd.concat([image_paths_df, labels], axis=1)

In [4]:
image_paths

[WindowsPath('data/images/apple_pie/1005649.jpg'),
 WindowsPath('data/images/apple_pie/1011328.jpg'),
 WindowsPath('data/images/apple_pie/101251.jpg'),
 WindowsPath('data/images/apple_pie/1014775.jpg'),
 WindowsPath('data/images/apple_pie/1026328.jpg'),
 WindowsPath('data/images/apple_pie/1028787.jpg'),
 WindowsPath('data/images/apple_pie/1034399.jpg'),
 WindowsPath('data/images/apple_pie/103801.jpg'),
 WindowsPath('data/images/apple_pie/1038694.jpg'),
 WindowsPath('data/images/apple_pie/1043283.jpg'),
 WindowsPath('data/images/apple_pie/1047447.jpg'),
 WindowsPath('data/images/apple_pie/1050519.jpg'),
 WindowsPath('data/images/apple_pie/1057749.jpg'),
 WindowsPath('data/images/apple_pie/1057810.jpg'),
 WindowsPath('data/images/apple_pie/1068632.jpg'),
 WindowsPath('data/images/apple_pie/1072416.jpg'),
 WindowsPath('data/images/apple_pie/1074856.jpg'),
 WindowsPath('data/images/apple_pie/1074942.jpg'),
 WindowsPath('data/images/apple_pie/1076891.jpg'),
 WindowsPath('data/images/apple_p

In [5]:
df

,imagepath,label
0,data\images\apple_pie\1005649.jpg,apple_pie
1,data\images\apple_pie\1011328.jpg,apple_pie
2,data\images\apple_pie\101251.jpg,apple_pie
3,data\images\apple_pie\1014775.jpg,apple_pie
4,data\images\apple_pie\1026328.jpg,apple_pie
...,...,...
4995,data\images\spaghetti_bolognese\980803.jpg,spaghetti_bolognese
4996,data\images\spaghetti_bolognese\989881.jpg,spaghetti_bolognese
4997,data\images\spaghetti_bolognese\990201.jpg,spaghetti_bolognese
4998,data\images\spaghetti_bolognese\991183.jpg,spaghetti_bolognese


In [6]:
df['label'].value_counts()

apple_pie              1000
chicken_wings          1000
fish_and_chips         1000
omelette               1000
spaghetti_bolognese    1000
Name: label, dtype: int64

In [7]:
train_df, test_df = train_test_split(df, train_size=0.8, shuffle=True)

In [12]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input
)

In [13]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='imagepath',
    y_col='label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=64,
    shuffle=True,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='imagepath',
    y_col='label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=64,
    shuffle=True,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='imagepath',
    y_col='label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=64,
    shuffle=False
)

Found 3200 validated image filenames belonging to 5 classes.
Found 800 validated image filenames belonging to 5 classes.
Found 1000 validated image filenames belonging to 5 classes.


In [21]:
resnet_model = ResNet50(weights='imagenet',input_shape=(224, 224, 3),
    include_top=False,
    pooling='avg')
resnet_model.trainable = False
inputs = resnet_model.input

x = tf.keras.layers.Dense(512, activation='relu')(resnet_model.output)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)

outputs = tf.keras.layers.Dense(5, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)


print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

                                                                                                  
 conv2_block3_1_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_2_conv (Conv2D)   (None, 56, 56, 64)   36928       ['conv2_block3_1_relu[0][0]']    
                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 56, 56, 64)  256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv3_block3_1_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_2_conv (Conv2D)   (None, 28, 28, 128)  147584      ['conv3_block3_1_relu[0][0]']    
                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 28, 28, 128)  512        ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv4_block2_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block2_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_2_conv (Conv2D)   (None, 14, 14, 256)  590080      ['conv4_block2_1_relu[0][0]']    
                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_2_conv[0][0]']    
 ization) 

 conv4_block5_1_conv (Conv2D)   (None, 14, 14, 256)  262400      ['conv4_block4_out[0][0]']       
                                                                                                  
 conv4_block5_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block5_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block5_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block5_2_conv (Conv2D)   (None, 14, 14, 256)  590080      ['conv4_block5_1_relu[0][0]']    
                                                                                                  
 conv4_blo

                                                                  'conv5_block1_3_bn[0][0]']      
                                                                                                  
 conv5_block1_out (Activation)  (None, 7, 7, 2048)   0           ['conv5_block1_add[0][0]']       
                                                                                                  
 conv5_block2_1_conv (Conv2D)   (None, 7, 7, 512)    1049088     ['conv5_block1_out[0][0]']       
                                                                                                  
 conv5_block2_1_bn (BatchNormal  (None, 7, 7, 512)   2048        ['conv5_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_1_relu (Activatio  (None, 7, 7, 512)   0           ['conv5_block2_1_bn[0][0]']      
 n)       

In [23]:
resnet_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

                                                                                                  
 conv2_block3_1_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_2_conv (Conv2D)   (None, 56, 56, 64)   36928       ['conv2_block3_1_relu[0][0]']    
                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 56, 56, 64)  256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv3_block3_1_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_2_conv (Conv2D)   (None, 28, 28, 128)  147584      ['conv3_block3_1_relu[0][0]']    
                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 28, 28, 128)  512        ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv4_block2_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block2_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_2_conv (Conv2D)   (None, 14, 14, 256)  590080      ['conv4_block2_1_relu[0][0]']    
                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_2_conv[0][0]']    
 ization) 

 conv4_block5_1_conv (Conv2D)   (None, 14, 14, 256)  262400      ['conv4_block4_out[0][0]']       
                                                                                                  
 conv4_block5_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block5_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block5_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block5_2_conv (Conv2D)   (None, 14, 14, 256)  590080      ['conv4_block5_1_relu[0][0]']    
                                                                                                  
 conv4_blo

                                                                  'conv5_block1_3_bn[0][0]']      
                                                                                                  
 conv5_block1_out (Activation)  (None, 7, 7, 2048)   0           ['conv5_block1_add[0][0]']       
                                                                                                  
 conv5_block2_1_conv (Conv2D)   (None, 7, 7, 512)    1049088     ['conv5_block1_out[0][0]']       
                                                                                                  
 conv5_block2_1_bn (BatchNormal  (None, 7, 7, 512)   2048        ['conv5_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_1_relu (Activatio  (None, 7, 7, 512)   0           ['conv5_block2_1_bn[0][0]']      
 n)       

In [22]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=10,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        )
    ]
)

Epoch 1/10
50/50 [==============================] - 149s 3s/step - loss: 0.3938 - accuracy: 0.8947 - val_loss: 0.2312 - val_accuracy: 0.9300
Epoch 2/10
50/50 [==============================] - 136s 3s/step - loss: 0.1068 - accuracy: 0.9675 - val_loss: 0.2392 - val_accuracy: 0.9300
Epoch 3/10
50/50 [==============================] - 149s 3s/step - loss: 0.0766 - accuracy: 0.9769 - val_loss: 0.1634 - val_accuracy: 0.9625
Epoch 4/10
50/50 [==============================] - 156s 3s/step - loss: 0.0425 - accuracy: 0.9859 - val_loss: 0.1926 - val_accuracy: 0.9463
Epoch 5/10
50/50 [==============================] - 157s 3s/step - loss: 0.0307 - accuracy: 0.9906 - val_loss: 0.2015 - val_accuracy: 0.9588
Epoch 6/10
50/50 [==============================] - 152s 3s/step - loss: 0.0118 - accuracy: 0.9972 - val_loss: 0.1764 - val_accuracy: 0.9700
Epoch 7/10
50/50 [==============================] - 140s 3s/step - loss: 0.0338 - accuracy: 0.9900 - val_loss: 0.1602 - val_accuracy: 0.9613
Epoch 8/10
50

In [23]:
# model.save('resnet50.h5')

INFO:tensorflow:Assets written to: model9997.file\assets


INFO:tensorflow:Assets written to: model9997.file\assets


In [24]:
test = tf.keras.models.load_model('resnet50.h5')

In [37]:
train_images[0][0][0].reshape(1,224,224,3)

array([[[[ 1.2060997e+01,  3.6221001e+01,  4.3320000e+01],
         [ 1.2060997e+01,  3.6221001e+01,  4.3320000e+01],
         [ 1.2060997e+01,  3.6221001e+01,  4.3320000e+01],
         ...,
         [ 3.0609970e+00,  3.2221001e+01,  3.7320000e+01],
         [ 3.0609970e+00,  3.0221001e+01,  3.5320000e+01],
         [ 6.0997009e-02,  2.7221001e+01,  3.2320000e+01]],

        [[ 9.0609970e+00,  3.3221001e+01,  4.0320000e+01],
         [ 1.6060997e+01,  4.0221001e+01,  4.7320000e+01],
         [ 1.3060997e+01,  3.7221001e+01,  4.4320000e+01],
         ...,
         [ 6.0997009e-02,  2.9221001e+01,  3.4320000e+01],
         [ 6.0997009e-02,  2.7221001e+01,  3.2320000e+01],
         [-2.9390030e+00,  2.4221001e+01,  2.9320000e+01]],

        [[ 1.1060997e+01,  3.5221001e+01,  4.2320000e+01],
         [ 1.4060997e+01,  3.8221001e+01,  4.5320000e+01],
         [ 1.7060997e+01,  4.1221001e+01,  4.8320000e+01],
         ...,
         [ 2.0609970e+00,  3.1221001e+01,  3.6320000e+01],
         [

In [39]:
np.argmax(test.predict(train_images[0][0][0].reshape(1,224,224,3)))

1/1 [==============================] - 0s 100ms/step


1

In [41]:
train_images[0][1]

array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0.

In [42]:
results = model.evaluate(test_images, verbose=0)
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

Test Accuracy: 94.90%


In [43]:
test_preds = np.argmax(model.predict(test_images), axis=1)

16/16 [==============================] - 37s 2s/step


In [47]:
print(classification_report(test_images.labels, test_preds, target_names=test_images.class_indices))

                     precision    recall  f1-score   support

          apple_pie       0.93      0.93      0.93       201
      chicken_wings       0.94      0.96      0.95       204
     fish_and_chips       0.97      0.95      0.96       228
           omelette       0.93      0.96      0.94       182
spaghetti_bolognese       0.97      0.96      0.96       185

           accuracy                           0.95      1000
          macro avg       0.95      0.95      0.95      1000
       weighted avg       0.95      0.95      0.95      1000



In [2]:
import tensorflow as tf
import numpy as np
from pathlib import Path
import pandas as pd

test = tf.keras.models.load_model('resnet50.h5')

image_dir = Path('images/')
pred_img_path = list(image_dir.glob(r'*.jpg'))
image_paths_df = pd.Series(pred_img_path, name='imagepath').astype(str)
fake_label = pd.Series('test', name='label')
pred_df = pd.concat([image_paths_df, fake_label], axis=1)

labels = {
    0 : 'apple_pie',
    1 : 'chicken_wings',
    2 : 'fish_and_chips',
    3 : 'omelette',
    4 : 'spaghetti_bolognese'
}

pred_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input,
)

pred_images = pred_generator.flow_from_dataframe(
    dataframe=pred_df,
    x_col='imagepath',
    y_col='label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=64,
    shuffle=True,
    subset='training'
)

pred_img = pred_images.next()[0]

prediction = np.argmax(test.predict(pred_img))
label = labels[prediction]
print(label)

Found 1 validated image filenames belonging to 1 classes.
1/1 [==============================] - 1s 1s/step
fish_and_chips
